This notebook was used to obtain the elongation values for the rooms in the S3DIS dataset.

In [ ]:
import open3d as o3d
import glob
import argparse
import numpy as np
from pathlib import Path    


In [ ]:

def get_bb(data):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(data)
    max_ = data.max(axis=0)
    min_ = data.min(axis=0)
    bb = pcd.get_oriented_bounding_box()
    return bb

def parse_crop_file(f):
    return np.loadtxt(f, usecols = (0,1,2))  

def get_elongation(bb):
    sorted_extents = sorted(bb.extent)
    # shortest axis over longest axis
    return sorted_extents[0] / sorted_extents[-1]
    


In [ ]:
stanford_dir = "/scratch/users/branisj/Stanford3dDataset_v1.2_Aligned_Version/"
areas = [f"Area_{x}" for x in range(1, 7)]
with open("elongations.csv", "w") as output_file:
    output_file.write("room,elongation\n")
    for area_number in range(1, 7):        
        area_name = f"Area_{area_number}"
        print(area_name)
        area_glob = stanford_dir + area_name + "/**/*.txt"
        files = sorted(glob.glob(area_glob))
        for file in files:
            print(f"Processing {file}")
            data = None
            with open(file) as f:
                data = parse_crop_file(f)
            bb = get_bb(data)
            elongation = get_elongation(bb)
            full_filename = area_name + "_" + Path(file).name.replace("txt", "ply")
            output_file.write(f"{full_filename},{elongation}\n")
